In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 35216480
paper_name = 'yoon_jun_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [20]:
original_data = pd.read_csv('raw_data/Table1.txt', header=None, names=['genes','perc'], sep=' ')

In [21]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 12 x 2


In [22]:
original_data.head()

,genes,perc
0,VPS9,89
1,VPS8,88
2,VPS21,76
3,VAM7,72
4,VPS3,71


In [23]:
original_data['genes'] = original_data['genes'].astype(str)

In [24]:
# Eliminate all white spaces & capitalize
original_data['genes'] = clean_genename(original_data['genes'])

In [25]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['genes'], to='orf')

In [26]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [genes, perc, orf]
Index: []


In [27]:
# WT cells have ~30% cells with symmetric distribution of PMA1 (i.e., they've lost the asymmetry)
# After normalization to 30%, negative values indicate less asymmetry (i.e., more symmetry)
original_data['data'] = 30-original_data['perc']

In [29]:
original_data.set_index('orf', inplace=True)

In [30]:
original_data = original_data[['data']].copy()

In [31]:
original_data = original_data.groupby(original_data.index).mean()

In [32]:
original_data.shape

(12, 1)

# Prepare the final dataset

In [33]:
data = original_data.copy()

In [34]:
dataset_ids = [22110]
datasets = datasets.reindex(index=dataset_ids)

In [35]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [36]:
data.head()

dataset_id,22110
data_type,value
orf,
YAL002W,-58
YDR080W,-26
YDR495C,-41
YGL124C,-23
YGL212W,-42


## Subset to the genes currently in SGD

In [37]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [38]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22110
,data_type,value
gene_id,orf,
2,YAL002W,-58
2207,YDR080W,-26
2621,YDR495C,-41
2805,YGL124C,-23
2893,YGL212W,-42


# Normalize

In [39]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [40]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [41]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22110           
data_type       value     valuez
gene_id orf                     
2       YAL002W   -58 -29.596748
2207    YDR080W   -26 -13.267508
2621    YDR495C   -41 -20.921839
2805    YGL124C   -23 -11.736642
2893    YGL212W   -42 -21.432128

# Print out

In [42]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [43]:
from IO.save_data_to_db3 import *

In [44]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 35216480...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  5.78it/s]


Updating the data_modified_on field...
